In [1]:
import sys
sys.path.append('../')

from rocket_rag.utils import *
from rocket_rag.agent import RagAgent

c:\Users\HAOXUAN\miniconda3\envs\agents2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load = '40kg'

In [3]:
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]
rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Randomly selected file: {if_ts_filename}')

if_rocket_feature = fit_transform(ts_filename=[if_ts_filename],
                                  field='current',
                                  smooth=True,
                                  smooth_ws=15,
                                  tolist=False,
                                  verbo=True)

2024-05-29 17:47:41.430 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points


2024-05-29 17:47:41.441 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 17:47:41.441 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Randomly selected file: ../data/inference/40kg\spalling1\spalling1_40_6_2.csv


In [4]:
agent = RagAgent(vs_dir=f'../store/nodes_{load}.pkl')

2024-05-29 17:47:45.719 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-05-29 17:47:46.115 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.
2024-05-29 17:47:46.446 | INFO     | rocket_rag.agent:_init_openai_client:67 - OpenAI API key found. OpenAI client Initialized.
2024-05-29 17:47:46.447 | INFO     | rocket_rag.agent:__init__:59 - RAG agent initialized.


In [5]:
rocket_feature = agent.get_rocket_feature(if_ts_filename)
print(f'Rocket feature shape: {rocket_feature.shape}')
fault_prediction, score = agent.get_fault_prediction()
print(f'Fault prediction: {fault_prediction}, Score: {score}')

2024-05-29 17:47:46.453 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 17:47:46.456 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 17:47:46.457 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Rocket feature shape: (1, 20000)
Fault prediction: ['spalling1_40_6_1'], Score: 1.0


In [6]:
agent.generate_fault_diagnosis_statement()
print(agent.fault_diagnosis_json)

{'fault_type': 'spalling', 'degradation_level': 1, 'retrieval_result(s)': 'spalling1_40_6_1', 'score': 1.0, 'distances': [], 'description': 'The actuator is experiencing surface damage in the ball-screw mechanism at a low level, affecting its smoothness and efficiency. Immediate attention is required to prevent further deterioration and ensure optimal performance.'}


In [7]:
agent.refine_fault_diagnosis_statement()
print(agent.refined_diagnosis_json)

{'fault_type': 'spalling', 'degradation_level': 1, 'refinement': 'No obvious fault detection'}


In [8]:
agent.generate_multi_queries()
print(agent.generated_queries)

['linear actuator surface damage repair', 'linear actuator ballscrew mechanism damage', 'how to fix ballscrew mechanism in linear actuator', 'linear actuator smoothness and efficiency restoration', 'prevent deterioration of linear actuator performance']


In [9]:
print(agent.tools.get_available_tools())

{'call_google': <function Tools.call_google at 0x00000148730F8900>, 'save_to_file': <function Tools.save_to_file at 0x00000148730F89A0>, 'get_available_tools': <function Tools.get_available_tools at 0x00000148730F8A40>}


In [10]:
agent.call_google_search()
print(agent.query_answers)

0it [00:00, ?it/s]2024-05-29 17:47:53.247 | INFO     | rocket_rag.agent:call_google_search:282 - Calling Google search API for query 1/5
0it [00:02, ?it/s]


BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[2].content', 'code': None}}